In [55]:
import json
import base64
import random
import requests
import numpy as np
from PIL import Image
from datetime import datetime
from cryptography.fernet import Fernet

import pywt

def generate_key():
    return Fernet.generate_key()

# Encrypt a message using symmetric encryption
def encrypt_message(message, key):
    cipher = Fernet(key)
    return cipher.encrypt(message.encode())

def decrypt_message(ciphertext, key):
    cipher = Fernet(key)
    return cipher.decrypt(ciphertext).decode()

def get_current_location():
    response = requests.get('https://ipinfo.io/json')
    data = response.json()
    city = data.get('city', '')
    region = data.get('region', '')
    country = data.get('country', '')
    loc = data.get('loc', '')  # latitude,longitude
    return {
        'city': city,
        'region': region,
        'country': country,
        'coordinates': loc
    }

## DWT Encoder

In [56]:
"""
Encodes a secret message into an image using DWT and saves the result.

Args:
    image_path (str): Path to the input image.
    secret_message (str): The message to hide.
    key (bytes): Encryption key for the message.
    output_path (str): Path to save the encoded image.
"""
def dwt_encode(image_path, secret_message, key, output_path):
    # Encrypt the message
    encrypted_message = encrypt_message(secret_message, key)
    # Convert encrypted message to base64 for easier embedding
    encoded_message = base64.urlsafe_b64encode(encrypted_message).decode()

    # Open image and convert to grayscale
    image = Image.open(image_path).convert('L')
    image_data = np.array(image)

    # Perform DWT (Discrete Wavelet Transform)
    coeffs = pywt.dwt2(image_data, 'haar')
    cA, (cH, cV, cD) = coeffs

    # Flatten cA and embed the message bits
    flat_cA = cA.flatten()
    # Convert to int for bitwise operations
    flat_cA_int = flat_cA.astype(np.int32)
    message_bits = ''.join(format(ord(char), '08b') for char in encoded_message)
    for i, bit in enumerate(message_bits):
        if i < len(flat_cA_int):
            flat_cA_int[i] = (flat_cA_int[i] & ~1) | int(bit)

    # Convert back to float for inverse DWT
    cA_embedded = flat_cA_int.astype(cA.dtype).reshape(cA.shape)
    coeffs_embedded = (cA_embedded, (cH, cV, cD))
    watermarked = pywt.idwt2(coeffs_embedded, 'haar')
    watermarked = np.clip(watermarked, 0, 255).astype(np.uint8)

    # Save the encoded image
    Image.fromarray(watermarked).save(output_path)

## DWT Decoder

In [57]:
"""
Decodes a secret message from an image encoded with DWT.

Args:
    image_path (str): Path to the encoded image.
    key (bytes): Encryption key for the message.

Returns:
    str: The decoded secret message.
"""
def dwt_decode(image_path, key):
    # Open image and convert to grayscale
    image = Image.open(image_path).convert('L')
    image_data = np.array(image)

    # Perform DWT (Discrete Wavelet Transform)
    coeffs = pywt.dwt2(image_data, 'haar')
    cA, (cH, cV, cD) = coeffs

    # Flatten cA and extract the message bits
    flat_cA = cA.flatten()
    bits = [str(int(val) & 1) for val in flat_cA]

    # Group bits into bytes and convert to characters
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) < 8:
            break
        chars.append(chr(int(''.join(byte), 2)))

    # Join characters and try to decode base64
    encoded_message = ''.join(chars)
    try:
        # Find the first padding character to trim extra bits
        end_idx = encoded_message.find('=')
        if end_idx != -1:
            encoded_message = encoded_message[:end_idx+2]
        encrypted_message = base64.urlsafe_b64decode(encoded_message)
        message = decrypt_message(encrypted_message, key)
        return message
    except Exception as e:
        return f"Decoding failed: {e}"

## Usage example

In [58]:
inputPath = "assets/images/raw.png"
outputPath = "assets/images/dwt-encoded.png"

secret_data = {
    "message": "Very Secret Message",
    "date": datetime.now().isoformat(),
    "location": get_current_location()
}

secret_message = json.dumps(secret_data, indent=4)

key = generate_key()
print("Generated encryption key (store this securely):")
print(key.decode())

# To encode:
dwt_encode(inputPath, secret_message, key, outputPath)

# To decode:
message = dwt_decode(outputPath, key)
print("\nDecoded message:")
print(message)

Generated encryption key (store this securely):
csx_74JkIOPd9-fVX4XgHYS2j6H73It3OroqLt45HqQ=

Decoded message:
Decoding failed: string argument should contain only ASCII characters
